<div class="alert alert-block alert-info" style="margin-top: 20px">
<h1 >Association Rule Learning</h1>
<p>Veri içerisindeki pattern&#39;leri (ilişkileri, yapıları) bulmak için kullanılan kural tabanlı bir makine öğrenmesi tekniğidir.</p>
<p><strong>Wallmart yaptığı analizlerde görüyor ki bebek bezi ile bira arasında bir ilişki var.</strong></p>
<p>Burada iki aksiyon fikri olabilir.</p>
<ul>
<li>İki ürünü farklı yerlere koymak. (ara raflarda satışı artırma hedefi)</li>
<li>Lift istatistiğine göre ürünleri yan yana koymak. (ikili satışı artırmak)</li>

</ul>
<p>Özetle birliktelik kuralları; veri seti içerisindeki basit istatistiksel örüntüleri çıkarır. Kabaca frekansları çıkarır.</p>
<p>Bu frekansları elde etmek mağazala raflarında çeşitli aksiyonlar alma fikirleri verir.</p>
<p>Aynı zamanda tavsiye sistemleri olarak da bunları kullanma imkanı verir.</p>
<p><strong>Ne işe yarar?</strong></p>
<ul>
<li>Pazarlama çalışmalarına katkı sağlar.</li>
<li>Marketlerin reyonlarının dizilimine katkı sağlar.</li>
<li>E-ticaret kategorilerinin dizaynlarına destek olur.</li>
<li>Çarpraz satışa destek olur. (mouse alana klavye önermek)</li>
<li>Ürün indirimine yardımcı olur.</li>
<li>Tavsiye sistemine yardımcı olur.</li>

</ul>
<p>En sık kullanılanı Apriori algoritmasıdır.</p>
</div>

<div class="alert alert-success" role="alert">
Amacımız retail_II veri setine birliktelik analizi uygulamak.

1. Veri Ön İşleme
   1. Eksik değer, aykırı değer vs (rfm'deki klasik işler)
   2. Transaction Datasından Invoice product (basket product) matrisini oluşturmak
2. Birliktelik Kurallarının Çıkarılması
</div>

## Data Preprocessing

In [7]:
# pip install mlxtend
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules
from helpers import *

In [2]:
df_ = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")
df = df_.copy()

In [6]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
check_df(df)

##################### Shape #####################
(541910, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00   2.55      17850.0  United Kingdom
1  536365     71053                 WHITE METAL LANTERN         6 2010-12-01 08:26:00   3.39      17850.0  United Kingdom
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00   2.75      17850.0  United Kingdom
##################### Tail #####################
       Invoice StockCode                      De

In [9]:
df = crm_data_prep(df)
check_df(df)

##################### Shape #####################
(397925, 9)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity         InvoiceDate  Price  Customer ID         Country  TotalPrice
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER       6.0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom       15.30
1  536365     71053                 WHITE METAL LANTERN       6.0 2010-12-01 08:26:00   3.39      17850.0  United Kingdom       20.34
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER       8.0 2010-12-01 08:26:00   2.75      17850.0  United Kingdom       22.00
###################

In [10]:
# Kolaylık olması açısından Fransa'yı filtreleyelim ve verinin boyutunu küçültelim.
df_fr = df[df['Country'] == "France"]
check_df(df_fr)

##################### Shape #####################
(8343, 9)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object
##################### Head #####################
   Invoice StockCode                 Description  Quantity         InvoiceDate  Price  Customer ID Country  TotalPrice
26  536370     22728   ALARM CLOCK BAKELIKE PINK      24.0 2010-12-01 08:45:00   3.75      12583.0  France        90.0
27  536370     22727   ALARM CLOCK BAKELIKE RED       24.0 2010-12-01 08:45:00   3.75      12583.0  France        90.0
28  536370     22726  ALARM CLOCK BAKELIKE GREEN      12.0 2010-12-01 08:45:00   3.75      12583.0  France        45.0
##################### Tail #####################
       Invoice StockCode        

In [13]:
# Invoice product (basket product) matrisini oluşturmak
# Amaç Invoice'ları tekilleştirmek, ve her bir ürünün quantitysini sum etmek.
df_fr.groupby(['Invoice', 'StockCode']).agg({"Quantity": "sum"}).head(25)

Quantity
Invoice StockCode          
536370  10002          48.0
        21035          18.0
        21724          12.0
        21731          24.0
        21791          24.0
        21883          24.0
        21913          12.0
        22326          24.0
        22492          36.0
        22540          24.0
        22544          24.0
        22629          24.0
        22631          24.0
        22659          24.0
        22661          20.0
        22726          12.0
        22727          24.0
        22728          24.0
        22900          24.0
        POST            3.0
536852  21786          24.0
        21791          12.0
        22539          24.0
        22544          24.0
        22549          12.0

In [16]:
# Burada her bir ürün tekilleşti fakat her bir fatura hala daha tekil değil.
# Bizim istediğimiz her bir satırda sadece bir tane fatura adı olsun, sütunlarda ürünler olsun,
# kesişimlerde de hangi faturalardan kaçar tane olduğunu istiyoruz.

# Bunun için unstack metodu kullanılır.
df_fr.groupby(['Invoice', 'StockCode']).agg({"Quantity": "sum"}).unstack().iloc[0:5, 0:5]

Quantity                        
StockCode    10002 10120 10125 10135 11001
Invoice                                   
536370        48.0   NaN   NaN   NaN   NaN
536852         NaN   NaN   NaN   NaN   NaN
536974         NaN   NaN   NaN   NaN   NaN
537065         NaN   NaN   NaN   NaN   NaN
537463         NaN   NaN   NaN   NaN   NaN

In [17]:
# Buradaki NA'ler normal. Bu Invoice Product matrisinde, her invoiceta her product yok.
# Fakat üstteki sütunda bütün olası pro ductlar var.
# Doğal dil işlemenin, tavsiye sistemlerinin en büyük problemlerinden birisi bu tarz seyrek matris problemleridir.

In [18]:
# Kontrol edelim.
df[(df["StockCode"] == 10002) & (df["Invoice"] == 536370)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
31,536370,10002,INFLATABLE POLITICAL GLOBE,48.0,2010-12-01 08:45:00,0.85,12583.0,France,40.8


In [19]:
# Bu NA'leri sıfır ile dolduralım.
df_fr.groupby(['Invoice', 'StockCode']).\
    agg({"Quantity": "sum"}).\
    unstack().fillna(0).iloc[0:5, 0:5]

Quantity                        
StockCode    10002 10120 10125 10135 11001
Invoice                                   
536370        48.0   0.0   0.0   0.0   0.0
536852         0.0   0.0   0.0   0.0   0.0
536974         0.0   0.0   0.0   0.0   0.0
537065         0.0   0.0   0.0   0.0   0.0
537463         0.0   0.0   0.0   0.0   0.0

In [20]:
# Burada görünen quantity değerini 1 ya da 0 olarak görmek istiyoruz.
# apply: satır ya da sütuna göre çalışır(axis), applymap tüm elemanlara çalışır.
df_fr.groupby(['Invoice', 'StockCode']).\
    agg({"Quantity": "sum"}).\
    unstack().fillna(0).\
    applymap(lambda x: 1 if x > 0 else 0).iloc[0:5, 0:5]

Quantity                        
StockCode    10002 10120 10125 10135 11001
Invoice                                   
536370           1     0     0     0     0
536852           0     0     0     0     0
536974           0     0     0     0     0
537065           0     0     0     0     0
537463           0     0     0     0     0

In [21]:
# Tüm işlemleri bir fonksiyon olarak atamak istersek
def create_invoice_product_df(dataframe):
    return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().\
        unstack().fillna(0). \
        applymap(lambda x: 1 if x > 0 else 0)

In [22]:
fr_inv_pro_df = create_invoice_product_df(df_fr)

In [23]:
fr_inv_pro_df.head()

Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK   NINE DRAWER OFFICE TIDY   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET   TRELLIS COAT RACK  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  12 EGG HOUSE PAINTED WOOD  12 MESSAGE CARDS WITH ENVELOPES  12 PENCIL SMALL TUBE WOODLAND  12 PENCILS SMALL TUBE RED RETROSPOT  12 PENCILS SMALL TUBE SKULL  12 PENCILS TALL TUBE POSY  12 PENCILS TALL TUBE RED RETROSPOT  12 PENCILS TALL TUBE WOODLAND  15CM CHRISTMAS GLASS BALL 20 LIGHTS  16 PIECE CUTLERY SET PANTRY DESIGN  18PC WOODEN CUTLERY SET DISPOSABLE  20 DOLLY PEGS RETROSPOT  200 RED + WHITE BENDY STRAWS  3 HOOK HANGER MAGIC GARDEN  3 PIECE SPACEBOY COOKIE CUTTER SET  3 RAFFIA RIBBONS 50'S CHRISTMAS   3 STRIPEY MICE FELTCRAFT  3 TIER CAKE TIN RED AND CREAM  3 TRADITIONAl BISCUIT CUTTERS  SET  36 DOILIES DOLLY GIRL  36 DOILIES VINTAGE CHRISTMAS  36 FOIL HEART CAKE CASES  36 FOIL STAR CAKE CASES   36 PENCILS TUBE RED RETROSPOT  36 PENCILS TUBE SKULLS  36 PENCILS TUBE WOODLAND  3D HEARTS  HONEYCOMB PAPER GARLAND  3D TRADITIONAL CHRISTMAS STICKERS  3D VINTAGE CHRISTMAS STICKERS   4 IVORY DINNER CANDLES SILVER FLOCK  4 PINK DINNER CANDLE SILVER FLOCK  4 TRADITIONAL SPINNING TOPS  5 HOOK HANGER MAGIC TOADSTOOL  5 HOOK HANGER RED MAGIC TOADSTOOL  6 GIFT TAGS 50'S CHRISTMAS   6 GIFT TAGS VINTAGE CHRISTMAS   6 RIBBONS EMPIRE    6 RIBBONS RUSTIC CHARM  6 ROCKET BALLOONS   60 CAKE CASES DOLLY GIRL DESIGN  60 CAKE CASES VINTAGE CHRISTMAS  60 TEATIME FAIRY CAKE CASES  6PC WOOD PLATE SET DISPOSABLE  72 SWEETHEART FAIRY CAKE CASES  A PRETTY THANK YOU CARD  ABC TREASURE BOOK BOX   ADULT APRON APPLE DELIGHT  ADVENT CALENDAR GINGHAM SACK  AGED GLASS SILVER T-LIGHT HOLDER  AIRLINE BAG VINTAGE JET SET BROWN  AIRLINE BAG VINTAGE JET SET RED  AIRLINE BAG VINTAGE JET SET WHITE  AIRLINE BAG VINTAGE TOKYO 78  AIRLINE BAG VINTAGE WORLD CHAMPION   AIRLINE LOUNGE,METAL SIGN  ALARM CLOCK BAKELIKE CHOCOLATE  ALARM CLOCK BAKELIKE GREEN  ALARM CLOCK BAKELIKE IVORY  ALARM CLOCK BAKELIKE ORANGE  ALARM CLOCK BAKELIKE PINK  ALARM CLOCK BAKELIKE RED   ALPHABET HEARTS STICKER SHEET  ALUMINIUM STAMPED HEART  AMETHYST CHUNKY BEAD BRACELET W STR  ANGEL DECORATION STARS ON DRESS  ANTIQUE ALL GLASS CANDLESTICK  ANTIQUE GLASS DRESSING TABLE POT  ANTIQUE GLASS PEDESTAL BOWL  ANTIQUE SILVER BAUBLE LAMP    ANTIQUE SILVER T-LIGHT GLASS  ANTIQUE SILVER TEA GLASS ENGRAVED  APPLE BATH SPONGE  AREA PATROLLED METAL SIGN  ASS COLOUR GLOWING TIARAS  ASS FLORAL PRINT MULTI SCREWDRIVER  ASSORTED BOTTLE TOP  MAGNETS   ASSORTED COLOUR BIRD ORNAMENT  ASSORTED COLOUR MINI CASES  ASSORTED COLOUR T-LIGHT HOLDER  ASSORTED COLOURS SILK FAN  ASSORTED EASTER DECORATIONS  BELLS  ASSORTED EASTER GIFT TAGS  ASSORTED FLOWER COLOUR "LEIS"  ASSORTED TUTTI FRUTTI BRACELET  ASSORTED TUTTI FRUTTI MIRROR  ASSORTED TUTTI FRUTTI SMALL PURSE  ASSTD DESIGN 3D PAPER STICKERS  ASSTD DESIGN RACING CAR PEN  ASSTD FRUIT+FLOWERS FRIDGE MAGNETS  ASSTD RASTA KEY-CHAINS  BABUSHKA LIGHTS STRING OF 10  BAG 125g SWIRLY MARBLES  BAG 250g SWIRLY MARBLES  BAKING MOULD CHOCOLATE CUPCAKES  BAKING MOULD EASTER EGG MILK CHOC  BAKING MOULD EASTER EGG WHITE CHOC  BAKING MOULD HEART MILK CHOCOLATE  BAKING MOULD HEART WHITE CHOCOLATE  BAKING SET 9 PIECE RETROSPOT   BAKING SET SPACEBOY DESIGN  BALLOON ART MAKE YOUR OWN FLOWERS  BALLOON PUMP WITH 10 BALLOONS  BALLOON WATER BOMB PACK OF 35  BALLOONS  WRITING SET   BANQUET BIRTHDAY  CARD    BASKET OF TOADSTOOLS  BATH BUILDING BLOCK WORD  BEADED CRYSTAL HEART BLUE  LARGE  BELLE JARDINIERE CUSHION COVER  BEWARE OF THE CAT METAL SIGN   BIG DOUGHNUT FRIDGE MAGNETS  BILI NUT AND WOOD NECKLACE  BINGO SET  BIRD DECORATION GREEN POLKADOT  BIRD DECORATION RED RETROSPOT  BIRD HOUSE HOT WATER BOTTLE  BIRDS MOBILE VINTAGE DESIGN  BIRTHDAY CARD, RETRO SPOT  BISCUIT TIN 50'S CHRISTMAS  BISCUIT TIN VINTAGE CHRISTMAS  BISCUIT TIN VINTAGE GREEN  BISCUIT TIN VINTAGE RED  BLACK AND WHITE CAT BOWL  BLACK CANDELABRA T-LIGHT HOLDER  BLACK CHRISTMAS TR

In [25]:
# Her bir invoice'da kaç eşsiz ürün vardır?
# df_fr.groupby("StockCode").apply(lambda x: x.nunique())["Invoice"].sort_values(ascending=False)
df_fr.groupby("StockCode").agg({"Invoice":"nunique"})


,Invoice
StockCode,
10002,8
10120,1
10125,6
10135,1
11001,1
...,...
90201B,2
90201C,1
C2,2


In [27]:
df_fr[df_fr["Invoice"]==536852]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
5252,536852,22549,PICTURE DOMINOES,12.0,2010-12-03 09:51:00,1.45,12686.0,France,17.40
5253,536852,22544,MINI JIGSAW SPACEBOY,24.0,2010-12-03 09:51:00,0.42,12686.0,France,10.08
5254,536852,22539,MINI JIGSAW DOLLY GIRL,24.0,2010-12-03 09:51:00,0.42,12686.0,France,10.08
5255,536852,22661,CHARLOTTE BAG DOLLY GIRL DESIGN,10.0,2010-12-03 09:51:00,0.85,12686.0,France,8.50
5256,536852,21791,VINTAGE HEADS AND TAILS CARD GAME,12.0,2010-12-03 09:51:00,1.25,12686.0,France,15.00
5257,536852,21786,POLKADOT RAIN HAT,24.0,2010-12-03 09:51:00,0.42,12686.0,France,10.08
5258,536852,POST,POSTAGE,1.0,2010-12-03 09:51:00,18.00,12686.0,France,18.00


In [28]:
# Her bir product kaç eşsiz sepettedir?
# df_fr.groupby("StockCode").apply(lambda x: x.nunique())["Invoice"].sort_values(ascending=False)
df_fr.groupby("StockCode").agg({"Invoice":"nunique"})


,Invoice
StockCode,
10002,8
10120,1
10125,6
10135,1
11001,1
...,...
90201B,2
90201C,1
C2,2


In [29]:
df_fr[df_fr["StockCode"]==10002]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
31,536370,10002,INFLATABLE POLITICAL GLOBE,48.0,2010-12-01 08:45:00,0.85,12583.0,France,40.8
21550,538093,10002,INFLATABLE POLITICAL GLOBE,12.0,2010-12-09 14:49:00,0.85,12682.0,France,10.2
23296,538196,10002,INFLATABLE POLITICAL GLOBE,36.0,2010-12-10 10:56:00,0.85,12731.0,France,30.6
44809,540239,10002,INFLATABLE POLITICAL GLOBE,12.0,2011-01-05 14:48:00,0.85,12682.0,France,10.2
64502,541631,10002,INFLATABLE POLITICAL GLOBE,12.0,2011-01-20 10:48:00,0.85,12637.0,France,10.2
76571,542629,10002,INFLATABLE POLITICAL GLOBE,120.0,2011-01-31 09:57:00,0.85,12731.0,France,102.0
77515,542735,10002,INFLATABLE POLITICAL GLOBE,12.0,2011-01-31 15:36:00,0.85,12681.0,France,10.2
142354,548606,10002,INFLATABLE POLITICAL GLOBE,120.0,2011-04-01 11:10:00,0.85,12731.0,France,102.0


### Birliktelik Kurallarının Çıkarılması

In [31]:
frequent_itemsets = apriori(fr_inv_pro_df, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False)

,support,itemsets
329,0.773779,(POSTAGE)
331,0.187661,(RABBIT NIGHT LIGHT)
370,0.179949,(RED TOADSTOOL LED NIGHT LIGHT)
319,0.172237,(PLASTERS IN TIN WOODLAND ANIMALS)
314,0.169666,(PLASTERS IN TIN CIRCUS PARADE )
...,...,...
5746,0.010283,"(PLASTERS IN TIN SPACEBOY, LUNCH BAG RED RETRO..."
20570,0.010283,"(DOLLY GIRL CHILDRENS CUP, SPACEBOY BIRTHDAY C..."
20569,0.010283,"(ROUND SNACK BOXES SET OF4 WOODLAND , DOLLY GI..."
20568,0.010283,"(SPACEBOY CHILDRENS BOWL, RED RETROSPOT CHARLO..."


In [32]:
# Belirlediğimiz bir thresholda göre kuralları çıkaracağız.
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(CHARLOTTE BAG DOLLY GIRL DESIGN),( DOLLY GIRL BEAKER),0.066838,0.023136,0.012853,0.192308,8.311966,0.011307,1.209450
1,( DOLLY GIRL BEAKER),(CHARLOTTE BAG DOLLY GIRL DESIGN),0.023136,0.066838,0.012853,0.555556,8.311966,0.011307,2.099614
2,(DOLLY GIRL CHILDRENS BOWL),( DOLLY GIRL BEAKER),0.046272,0.023136,0.017995,0.388889,16.808642,0.016924,1.598504
3,( DOLLY GIRL BEAKER),(DOLLY GIRL CHILDRENS BOWL),0.023136,0.046272,0.017995,0.777778,16.808642,0.016924,4.291774
4,(DOLLY GIRL CHILDRENS CUP),( DOLLY GIRL BEAKER),0.041131,0.023136,0.015424,0.375000,16.208333,0.014473,1.562982


In [33]:
rules.sort_values("lift", ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1133756,"(ALARM CLOCK BAKELIKE RED , LUNCH BAG APPLE DE...","(LUNCH BOX I LOVE LONDON, JUMBO BAG APPLES, LU...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1230093,"(ALARM CLOCK BAKELIKE RED , LUNCH BOX WITH CUT...","(JUMBO BAG APPLES, ALARM CLOCK BAKELIKE PINK, ...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1230087,"(ALARM CLOCK BAKELIKE RED , JUMBO BAG APPLES, ...","(LUNCH BOX WITH CUTLERY RETROSPOT , LUNCH BAG ...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1230088,"(ALARM CLOCK BAKELIKE RED , JUMBO BAG APPLES, ...","(LUNCH BOX WITH CUTLERY RETROSPOT , LUNCH BAG ...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1230089,"(ALARM CLOCK BAKELIKE RED , JUMBO BAG APPLES, ...","(LUNCH BOX WITH CUTLERY RETROSPOT , LUNCH BAG ...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf


<div class="alert alert-success" role="alert">

**antecedent support:** Tek başına görülme olasılığı

**Support**: iki ürünün birlikte gözlenme olasılığı

**Confidence**: bir ürün alındığında diğerinin alınma olasılığı.

**Lift**: bir ürün alındığında diğer ürünün alınma olasılığı "lift" kat artar anlamına gelir.

**Conviction**: Y olmadan X'in beklenen frekansı

**Leverage**: Aslında lifte benzer bir değer. Supportu yüksek ögelere öncelik verme eğilimindedir.

**Lift**: ise daha az sıklığa rağmen güçlü ilişkileri bulabilmektedir.
</div>

<div class="alert alert-warning" role="alert">
<strong>Buradan neler yapılabilir?</strong>

Segmentlere göre özel kurallar çıkarılıp ayrı ayrı önerilerde bulunabilir.( Davranışlarının farklı olduğunu varsayarak.) 

Örn: CLTV segmentlerininin sepet analizi yapılıp, ürün tavsiyeleri yapılabilir.
</div>